# Translating (JUNOS)

As explained in the previous tutorial, a ntc_rosetta driver can both parse and translate between native and yang-based models. In this tutorial we are going to translate data that complies to the openconfig model into native Junos \(XML\) configuration.

Let's start by loading the needed driver:

In [1]:
from ntc_rosetta import get_driver

junos = get_driver("junos", "openconfig")
junos_processor = junos()

Now we need some data:

In [2]:
data = {
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "name": "xe-0/0/1",
                "config": {
                    "name": "xe-0/0/1",
                    "type": "iana-if-type:ethernetCsmacd",
                    "description": "This is xe-0/0/1",
                    "enabled": False
                },
                "subinterfaces": {
                    "subinterface": [
                        {
                            "index": 1,
                            "config": {
                                "index": 1,
                                "description": "This is xe-0/0/1.1"
                            }
                        },
                        {
                            "index": 2,
                            "config": {
                                "index": 2,
                                "description": "This is xe-0/0/1.2"
                            }
                        }
                    ]
                }
            },
            {
                "name": "xe-0/0/2",
                "config": {
                    "name": "xe-0/0/2",
                    "type": "iana-if-type:ethernetCsmacd",
                    "description": "This is xe-0/0/2",
                    "enabled": True
                },
                "openconfig-if-ethernet:ethernet": {
                    "openconfig-vlan:switched-vlan": {
                        "config": {
                            "interface-mode": "ACCESS",
                            "access-vlan": 10
                        }
                    }
                }
            },
            {
                "name": "xe-0/0/3",
                "config": {
                    "name": "xe-0/0/3",
                    "type": "iana-if-type:ethernetCsmacd",
                    "enabled": False
                },
                "openconfig-if-ethernet:ethernet": {
                    "openconfig-vlan:switched-vlan": {
                        "config": {
                            "interface-mode": "TRUNK",
                            "trunk-vlans": [
                                10,
                                20
                            ]
                        }
                    }
                }
            }
        ]
    },
    "openconfig-network-instance:network-instances": {
        "network-instance": [
            {
                "name": "default",
                "config": {
                    "name": "default"
                },
                "vlans": {
                    "vlan": [
                        {
                            "vlan-id": 10,
                            "config": {
                                "vlan-id": 10,
                                "name": "prod",
                                "status": "ACTIVE"
                            }
                        },
                        {
                            "vlan-id": 20,
                            "config": {
                                "vlan-id": 20,
                                "name": "dev",
                                "status": "SUSPENDED"
                            }
                        }
                    ]
                }
            }
        ]
    }
}

Once we have the data, translating it to native is very simple:

In [3]:
native = junos_processor.translate(candidate=data)

We can verify the result by just printing it:

In [4]:
print(native)

<configuration>
  <interfaces>
    <interface>
      <name>xe-0/0/1</name>
      <disable/>
      <unit>
        <name>1</name>
        <description>This is xe-0/0/1.1</description>
      </unit>
      <unit>
        <name>2</name>
        <description>This is xe-0/0/1.2</description>
      </unit>
    </interface>
    <interface>
      <name>xe-0/0/2</name>
      <disable delete="delete"/>
      <unit>
        <name>0</name>
        <family>
          <ethernet-switching>
            <interface-mode>access</interface-mode>
            <vlan>
              <members>10</members>
            </vlan>
          </ethernet-switching>
        </family>
      </unit>
    </interface>
    <interface>
      <name>xe-0/0/3</name>
      <disable/>
      <unit>
        <name>0</name>
        <family>
          <ethernet-switching>
            <interface-mode>trunk</interface-mode>
            <vlan>
              <members>10</members>
              <members>20</members>
            </vlan>
       